# Competition: [30 Days of ML | Kaggle](https://www.kaggle.com/c/30-days-of-ml)

- Version: v6 - Hyperparameter tuning
- Description: Notebook 30 Days -v6- Hyperparameter tuning | Model v6.12
- Validation Score: - 0.7178869096515929
- Public Score: 0.71908

In [5]:
submission_csv = 'submission_v6.12-hyperparam-tuning.csv'

# Step 1: Import helpful libraries

In [6]:
# Familiar imports
import numpy as np
import pandas as pd

# For ordinal encoding categorical variables, splitting data
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# For Hyperparameter tuning
import optuna

# For training models
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Step 2: Load the data

In [7]:
# Absorb the difference in directory structure between the Kaggle kernel and the local environment
import os
is_kaggle = 'KAGGLE_URL_BASE' in set(os.environ.keys())
if is_kaggle:
    compe_name = os.listdir(path='../input')[0]
    input_train_csv_path = '../input/30days-folds/train_folds.csv'
    input_test_csv_path = '../input/' + compe_name + '/test.csv'
    input_sample_submission_csv_path = '../input/' + compe_name + '/sample_submission.csv'
    output_dir_path = ''
else:
    input_train_csv_path = '../output/dataset_v0_train-folds.csv'
    input_test_csv_path = '../input/test.csv'
    input_sample_submission_csv_path = '../input/sample_submission.csv'
    output_dir_path = '../output/'

print('input_train_csv_path:',input_train_csv_path)
print('input_test_csv_path:',input_test_csv_path)
print('input_sample_submission_csv_path:',input_sample_submission_csv_path)
print('output_dir_path',output_dir_path)

input_train_csv_path: ../output/dataset_v0_train-folds.csv
input_test_csv_path: ../input/test.csv
input_sample_submission_csv_path: ../input/sample_submission.csv
output_dir_path ../output/


In [8]:
# Load the training data
X_full = pd.read_csv(input_train_csv_path)
X_test_full = pd.read_csv(input_test_csv_path)
X_sample_submission = pd.read_csv(input_sample_submission_csv_path)

# Preview the data
X_full.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
0,1,B,B,B,C,B,B,A,E,C,...,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,0
1,2,B,B,A,A,B,D,A,F,A,...,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,2
2,3,A,A,A,C,B,D,A,D,A,...,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4
3,4,B,B,A,C,B,D,A,E,C,...,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,3
4,6,A,A,A,C,B,D,A,E,A,...,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,1


## Naming rules fof data variables

- `X_full`: Training + validation data (full columns) → `X`: training + validation data (selected columns) ｜ `y`: training + validation labels
  - `X_train_full`: Training data (full columns) → `X_train_sel`: Training data (selected columns) → `X_train`: Training data (selected columns + OneHot) ｜ `y_train`: Training labels
  - `X_valid_full`: Validation data (full columns) → `X_valid_sel`: Validation data (selected columns) → `X_valid`: Validation data (selected columns + OneHot) ｜ `y_valid`: Validation labels
- `X_test_full`: Test data (full columns) → `X_test_sel`: Test data (selected columns) → `X_test`: Test data (selected columns + OneHot) ｜ `y_test`: Test labels
- `X_sample_submission`: Samaple submission data

# Step 3: Prepare the data

In [9]:
# Check missing values
cols_with_missing = [col for col in X_full.columns if X_full[col].isnull().any()] 
print('cols_with_missing of train data:', cols_with_missing)

cols_with_missing = [col for col in X_test_full.columns if X_test_full[col].isnull().any()] 
print('cols_with_missing of test data:', cols_with_missing)

cols_with_missing of train data: []
cols_with_missing of test data: []


In [10]:
useful_features = [cname for cname in X_full.columns if cname not in ("id", "target", "kfold")]

#categorical_cols = [cname for cname in useful_features if df_train[cname].dtype == "object" and df_train[cname].nunique() < 10] # low-cardinality object cols
categorical_cols = [cname for cname in useful_features if X_full[cname].dtype == "object"] # low-cardinality object cols
numerical_cols = [cname for cname in useful_features if X_full[cname].dtype in ['int64', 'float64']] # [cname for cname in useful_features if 'cont' in cname]
print('categorical_cols:', categorical_cols)
print('numerical_cols:', numerical_cols)

useful_features = categorical_cols + numerical_cols
print('useful_features-prev:', useful_features)
print('useful_features-post:', useful_features)

categorical_cols: ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
numerical_cols: ['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13']
useful_features-prev: ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13']
useful_features-post: ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13']


In [11]:
X_test_sel = X_test_full[useful_features]
X_test_sel.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,C,B,B,A,E,E,I,...,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702
1,A,B,A,C,B,C,A,E,C,H,...,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940
2,B,A,A,A,B,B,A,E,D,K,...,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099
3,B,B,A,C,B,D,A,E,A,N,...,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372
4,B,B,A,C,B,C,A,E,C,F,...,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412


# Step 4: Hyperparameter tuning

In [15]:
def run(trial):
    #print('fold=', trial.number)
    fold = 0
    
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 7)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 20)

    X_train = X_full[X_full.kfold != fold].reset_index(drop=True)
    X_valid = X_full[X_full.kfold == fold].reset_index(drop=True)

    y_train = X_train.target
    y_valid = X_valid.target
    
    X_train = X_train[useful_features]
    X_valid = X_valid[useful_features]
    
    onehot_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    X_train_ohe = onehot_encoder.fit_transform(X_train[categorical_cols])
    X_valid_ohe = onehot_encoder.transform(X_valid[categorical_cols])

    df_train_ohe = pd.DataFrame(X_train_ohe, columns=[f'ohe_{i}' for i in range(X_train_ohe.shape[1])])
    df_valid_ohe = pd.DataFrame(X_valid_ohe, columns=[f'ohe_{i}' for i in range(X_valid_ohe.shape[1])])

    X_train = X_train.drop(categorical_cols, axis=1)
    X_valid = X_valid.drop(categorical_cols, axis=1)

    X_train = pd.concat([X_train, df_train_ohe], axis=1)
    X_valid = pd.concat([X_valid, df_valid_ohe], axis=1)

    scaler = StandardScaler()
    X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
    X_valid[numerical_cols] = scaler.transform(X_valid[numerical_cols])

    #model = RandomForestRegressor(random_state=fold, n_jobs=-1) # verbose =10, 
    model = XGBRegressor(
        # verbosity =1, 
        #n_jobs=-1, # To use CPU
        n_estimators=10000, # No need to optimize because of early_stopping

        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
        min_child_weight=min_child_weight,

        tree_method='gpu_hist', gpu_id=-1, predictor='gpu_predictor', # To use GPU
        random_state=42) 
    model.fit(
        X_train, y_train,
        early_stopping_rounds=300, eval_set=[(X_valid, y_valid)],
        verbose=1000) # Output every 1000 estimators

    preds_valid = model.predict(X_valid)
    score_valid = mean_squared_error(y_valid, preds_valid, squared=False)
    return score_valid

In [16]:
study = optuna.create_study(direction="minimize")
study.optimize(run, n_trials=500)

[I 2021-08-30 23:51:39,539] A new study created in memory with name: no-name-4ff79c63-0821-4b7e-9f4d-f3f69c31faf4


[0]	validation_0-rmse:6.76737
[1000]	validation_0-rmse:0.72021
[1287]	validation_0-rmse:0.72029


[I 2021-08-30 23:51:48,399] Trial 0 finished with value: 0.7201335185422689 and parameters: {'learning_rate': 0.13150700508910687, 'reg_lambda': 1.19836590979346e-07, 'reg_alpha': 1.6441487953748144e-07, 'subsample': 0.34007741023404503, 'colsample_bytree': 0.3523945120386145, 'max_depth': 3, 'min_child_weight': 12}. Best is trial 0 with value: 0.7201335185422689.


[0]	validation_0-rmse:7.67247
[1000]	validation_0-rmse:0.72450
[2000]	validation_0-rmse:0.72091
[3000]	validation_0-rmse:0.71949
[4000]	validation_0-rmse:0.71881
[5000]	validation_0-rmse:0.71859
[6000]	validation_0-rmse:0.71845
[6377]	validation_0-rmse:0.71844


[I 2021-08-30 23:52:56,481] Trial 1 finished with value: 0.7184356173398119 and parameters: {'learning_rate': 0.013935688800258727, 'reg_lambda': 1.1471069466349636e-08, 'reg_alpha': 7.923963344221972, 'subsample': 0.9926219743843415, 'colsample_bytree': 0.5582431900124715, 'max_depth': 5, 'min_child_weight': 11}. Best is trial 1 with value: 0.7184356173398119.


...

[I 2021-08-31 04:13:48,744] Trial 497 finished with value: 0.7170204214621613 and parameters: {'learning_rate': 0.08671394577915055, 'reg_lambda': 5.032448298799328e-06, 'reg_alpha': 33.50965517166009, 'subsample': 0.9347032873866522, 'colsample_bytree': 0.12698108166604868, 'max_depth': 2, 'min_child_weight': 20}. Best is trial 425 with value: 0.7169728273961298.


[0]	validation_0-rmse:7.20205
[1000]	validation_0-rmse:0.72380
[2000]	validation_0-rmse:0.72058
[3000]	validation_0-rmse:0.71901
[4000]	validation_0-rmse:0.71809
[5000]	validation_0-rmse:0.71765
[6000]	validation_0-rmse:0.71740
[7000]	validation_0-rmse:0.71727
[8000]	validation_0-rmse:0.71722
[8939]	validation_0-rmse:0.71718


[I 2021-08-31 04:14:40,279] Trial 498 finished with value: 0.7171741630057654 and parameters: {'learning_rate': 0.07501341758359555, 'reg_lambda': 1.4881457406448892e-05, 'reg_alpha': 40.51824093254942, 'subsample': 0.9418461211200005, 'colsample_bytree': 0.13200535694922116, 'max_depth': 2, 'min_child_weight': 20}. Best is trial 425 with value: 0.7169728273961298.


[0]	validation_0-rmse:7.27888
[1000]	validation_0-rmse:0.72454
[2000]	validation_0-rmse:0.72127
[3000]	validation_0-rmse:0.71968
[4000]	validation_0-rmse:0.71865
[5000]	validation_0-rmse:0.71813
[6000]	validation_0-rmse:0.71781
[7000]	validation_0-rmse:0.71763
[8000]	validation_0-rmse:0.71751
[9000]	validation_0-rmse:0.71744
[9999]	validation_0-rmse:0.71740


[I 2021-08-31 04:15:38,889] Trial 499 finished with value: 0.7173965981369648 and parameters: {'learning_rate': 0.06503465061490189, 'reg_lambda': 0.07236405788580454, 'reg_alpha': 59.71215259319541, 'subsample': 0.982704648767124, 'colsample_bytree': 0.15300664127235253, 'max_depth': 2, 'min_child_weight': 20}. Best is trial 425 with value: 0.7169728273961298.


In [17]:
params = study.best_params
params

{'learning_rate': 0.11793269529467006,
 'reg_lambda': 6.243461856118161e-07,
 'reg_alpha': 21.796577223512983,
 'subsample': 0.9057108787852421,
 'colsample_bytree': 0.11575062453379714,
 'max_depth': 2,
 'min_child_weight': 19}

# Step 5: Train a model

In [ ]:
params = {
    'learning_rate': 0.11793269529467006,
    'reg_lambda': 6.243461856118161e-07,
    'reg_alpha': 21.796577223512983,
    'subsample': 0.9057108787852421,
    'colsample_bytree': 0.11575062453379714,
    'max_depth': 2,
    'min_child_weight': 19
    }

In [18]:
# Load the training data
X_full = pd.read_csv(input_train_csv_path)
X_test_full = pd.read_csv(input_test_csv_path)
useful_features = [cname for cname in X_full.columns if cname not in ("id", "target", "kfold")]
categorical_cols = [cname for cname in useful_features if X_full[cname].dtype == "object"] # low-cardinality object cols
X_test_sel = X_test_full[useful_features]

In [19]:
valid_scores = []
test_predictions = []

for fold in range(5):
    X_train =  X_full[X_full.kfold != fold].reset_index(drop=True)
    X_valid = X_full[X_full.kfold == fold].reset_index(drop=True)
    X_test = X_test_sel.copy()

    y_train = X_train.target
    y_valid = X_valid.target
    
    X_train = X_train[useful_features]
    X_valid = X_valid[useful_features]
    
    onehot_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    X_train_ohe = onehot_encoder.fit_transform(X_train[categorical_cols])
    X_valid_ohe = onehot_encoder.transform(X_valid[categorical_cols])
    X_test_ohe = onehot_encoder.transform(X_test[categorical_cols])

    df_train_ohe = pd.DataFrame(X_train_ohe, columns=[f'ohe_{i}' for i in range(X_train_ohe.shape[1])])
    df_valid_ohe = pd.DataFrame(X_valid_ohe, columns=[f'ohe_{i}' for i in range(X_valid_ohe.shape[1])])
    df_test_ohe = pd.DataFrame(X_test_ohe, columns=[f'ohe_{i}' for i in range(X_test_ohe.shape[1])])

    X_train = X_train.drop(categorical_cols, axis=1)
    X_valid = X_valid.drop(categorical_cols, axis=1)
    X_test = X_test.drop(categorical_cols, axis=1)

    X_train = pd.concat([X_train, df_train_ohe], axis=1)
    X_valid = pd.concat([X_valid, df_valid_ohe], axis=1)
    X_test = pd.concat([X_test, df_test_ohe], axis=1)

    scaler = StandardScaler()
    X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
    X_valid[numerical_cols] = scaler.transform(X_valid[numerical_cols])
    X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])
    
    #model = RandomForestRegressor(random_state=fold, n_jobs=-1) # verbose =10, 
    model = XGBRegressor(
        # verbosity =1, 
        #n_jobs=-1, # To use CPU

        n_estimators=10000,

        **params,

        tree_method='gpu_hist', gpu_id=-1, predictor='gpu_predictor', # To use GPU
        random_state=24) 
    model.fit(X_train, y_train, early_stopping_rounds=300, eval_set=[(X_valid, y_valid)], verbose=1000)

    preds_valid = model.predict(X_valid)
    score_valid = mean_squared_error(y_valid, preds_valid, squared=False)
    valid_scores.append(score_valid)
    print(fold, score_valid)

    preds_test = model.predict(X_test)
    test_predictions.append(preds_test)

[0]	validation_0-rmse:6.87165
[1000]	validation_0-rmse:0.72158
[2000]	validation_0-rmse:0.71872
[3000]	validation_0-rmse:0.71775
[4000]	validation_0-rmse:0.71731
[5000]	validation_0-rmse:0.71718
[5992]	validation_0-rmse:0.71715
0 0.7171132203018182
[0]	validation_0-rmse:6.86834
[1000]	validation_0-rmse:0.72093
[2000]	validation_0-rmse:0.71824
[3000]	validation_0-rmse:0.71739
[4000]	validation_0-rmse:0.71708
[4160]	validation_0-rmse:0.71708
1 0.7170617285468162
[0]	validation_0-rmse:6.86603
[1000]	validation_0-rmse:0.72298
[2000]	validation_0-rmse:0.72017
[3000]	validation_0-rmse:0.71926
[4000]	validation_0-rmse:0.71897
[4839]	validation_0-rmse:0.71892
2 0.7189069472481392
[0]	validation_0-rmse:6.86838
[1000]	validation_0-rmse:0.72284
[2000]	validation_0-rmse:0.72020
[3000]	validation_0-rmse:0.71928
[4000]	validation_0-rmse:0.71897
[5000]	validation_0-rmse:0.71893
[5228]	validation_0-rmse:0.71893
3 0.7189112999211044
[0]	validation_0-rmse:6.87396
[1000]	validation_0-rmse:0.72185
[2000]	

In [20]:
score_validation = np.mean(valid_scores)
print(score_validation)

0.7178869096515929


# Step 6: Submit to the competition

In [21]:
preds_submission = np.mean(np.column_stack(test_predictions), axis=1)
preds_submission

array([8.093393 , 8.399588 , 8.417988 , ..., 8.477141 , 8.150858 ,
       7.9716635], dtype=float32)

In [22]:
X_sample_submission.target = preds_submission
X_sample_submission.to_csv(output_dir_path + submission_csv, index=False)